## 1. scraping web

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager  
from bs4 import BeautifulSoup
import os
import requests
from PIL import Image
import time
from urllib.parse import urljoin
from datetime import datetime


# === CONFIG ===
service = Service("../webdriver/chromedriver-win64/chromedriver.exe")

timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M")
dump_path = f'../data/Go_Wholesale/{timestamp}'
output_folder = os.path.join(dump_path, "downloaded_pages")
output_pdf = os.path.join(dump_path, "catalog.pdf")
os.makedirs(dump_path, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

# === SETUP ===
options = Options()
options.headless = True
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# === STEP 1: เปิดเว็บหลัก ===
driver.get("https://centralfoodwholesale.co.th/e-catalogs/bi-weekly/")
time.sleep(2)  # รอโหลด

iframe = driver.find_element("css selector", "div.wpb_wrapper iframe")
flipbook_url = iframe.get_attribute("src") or iframe.get_attribute("data-lazy-src")
print(f"✅ Found flipbook URL: {flipbook_url}")

# === STEP 2: เปิด flipbook ===
driver.get(flipbook_url + "#p=1")
time.sleep(3)  # รอโหลด JS flipbook

# === STEP 3: Scroll ทีละหน้าและเก็บ <img>
image_urls = []
seen = set()
max_pages = 30
not_found_count = 0

for p in range(1, max_pages+1):
    driver.get(f"{flipbook_url}#p={p}")
    time.sleep(2)  # รอหน้าโหลด

    soup = BeautifulSoup(driver.page_source, "html.parser")
    found = False
    for img in soup.select("div.side-image img"):
        src = img.get("src")
        if src and ".webp" in src:
            # ✅ robust ใช้ urljoin ต่อ src ให้เป็น absolute URL
            full_src = urljoin(flipbook_url, src)
            if full_src not in seen:
                seen.add(full_src)
                image_urls.append(full_src)
                found = True

    print(f"📄 Page {p}: {'found' if found else 'no image'}")
    if not found:
        not_found_count += 1
        if not_found_count > 2:
            break
    else:
        not_found_count = 0

print(f"🎉 Found {len(image_urls)} images")

# === STEP 4: Download ===
for url in image_urls:
    fname = url.split("/")[-1].split("?")[0]
    print(f"⬇️ {url}")
    r = requests.get(url)
    with open(os.path.join(output_folder, fname), "wb") as f:
        f.write(r.content)

# === STEP 5: Merge PDF ===
imgs = [
    Image.open(os.path.join(output_folder, f)).convert("RGB")
    for f in sorted(os.listdir(output_folder))
    if f.endswith(".webp")
]
if imgs:
    imgs[0].save(output_pdf, save_all=True, append_images=imgs[1:])
    print("✅ Done: output_catalog.pdf")
else:
    print("⚠️ No images found to merge!")

driver.quit()

✅ Found flipbook URL: https://online.fliphtml5.com/ilurt/hdnm/
📄 Page 1: found
📄 Page 2: found
📄 Page 3: no image
📄 Page 4: found
📄 Page 5: no image
📄 Page 6: found
📄 Page 7: no image
📄 Page 8: found
📄 Page 9: no image
📄 Page 10: found
📄 Page 11: no image
📄 Page 12: found
📄 Page 13: no image
📄 Page 14: found
📄 Page 15: no image
📄 Page 16: no image
📄 Page 17: no image
🎉 Found 17 images
⬇️ https://online.fliphtml5.com/ilurt/hdnm/files/large/679098e2182817630b8d8cc022a99015.webp?1761036984&1761036984
⬇️ https://online.fliphtml5.com/ilurt/hdnm/files/large/14872ca2420c689524c014609c9e7389.webp?1761036984&1761036984
⬇️ https://online.fliphtml5.com/ilurt/hdnm/files/large/caa430029d1d4ad6e13b08c2115104f1.webp?1761036984&1761036984
⬇️ https://online.fliphtml5.com/ilurt/hdnm/files/thumb/40cd663469d9da211f05a0097645e6d6.webp?1761036984&1761036984
⬇️ https://online.fliphtml5.com/ilurt/hdnm/files/large/25df8d7823b61c6cfcf956c54cef2d2d.webp?1761036984&1761036984
⬇️ https://online.fliphtml5.com/ilurt

## 2.train yolo

In [4]:
import os
import shutil
import random
from ultralytics import YOLO

# ================= CONFIG =================
# โฟลเดอร์ที่ตอนนี้มีรูปกับ txt ปนกันอยู่
DATASET_ROOT = r"C:\projecteve\OCR\dataset\CJmall_catalog\obj_Train_data"

# โฟลเดอร์ปลายทางที่เราจะจัดเป็นโครงสร้างแบบ YOLO
OUTPUT_ROOT = r"C:\projecteve\OCR\dataset\CJmall_catalog\yolo"

TRAIN_RATIO = 0.8          # 80% train, 20% val
RANDOM_SEED = 42

# ไฟล์ data.yaml ที่จะให้ YOLO ใช้
DATA_YAML_PATH = os.path.join(OUTPUT_ROOT, "data.yaml")

# จำนวนคลาสและชื่อคลาส (แก้ให้ตรงกับของ Eve)
NUM_CLASSES = 1
CLASS_NAMES = ["product"]   # ถ้ามีหลายคลาสเช่น ['product', 'price_tag'] ก็ใส่เพิ่มได้

# โมเดล base ที่จะใช้เทรน
BASE_MODEL = "yolov8m.pt"   # เริ่มจากตัวเล็กก่อน ถ้ารอดค่อยขยับเป็น yolov8s/m


def ensure_dirs():
    """สร้างโฟลเดอร์ train/val + images/labels"""
    for split in ["train", "val"]:
        os.makedirs(os.path.join(OUTPUT_ROOT, split, "images"), exist_ok=True)
        os.makedirs(os.path.join(OUTPUT_ROOT, split, "labels"), exist_ok=True)


def collect_pairs():
    """
    มองหารูปกับ .txt ที่ชื่อเดียวกันใน DATASET_ROOT
    แล้วคืนลิสต์ (img_path, label_path)
    """
    exts = (".jpg", ".jpeg", ".png", ".bmp", ".webp")
    files = os.listdir(DATASET_ROOT)

    image_label_pairs = []

    for fname in files:
        if fname.lower().endswith(exts):
            basename = os.path.splitext(fname)[0]
            img_path = os.path.join(DATASET_ROOT, fname)
            label_path = os.path.join(DATASET_ROOT, basename + ".txt")
            if os.path.exists(label_path):
                image_label_pairs.append((img_path, label_path))

    return image_label_pairs


def split_dataset(pairs, train_ratio=0.8, seed=42):
    """สุ่มแบ่ง train/val โดยไม่ต้องใช้ sklearn"""
    random.Random(seed).shuffle(pairs)
    n_train = int(len(pairs) * train_ratio)
    train_pairs = pairs[:n_train]
    val_pairs = pairs[n_train:]
    return train_pairs, val_pairs


def copy_split(pairs, split):
    """ก็อปปี้รูปและ label ไปไว้ในโฟลเดอร์ split (train/val)"""
    for img_path, label_path in pairs:
        img_name = os.path.basename(img_path)
        lbl_name = os.path.basename(label_path)

        dst_img = os.path.join(OUTPUT_ROOT, split, "images", img_name)
        dst_lbl = os.path.join(OUTPUT_ROOT, split, "labels", lbl_name)

        shutil.copy(img_path, dst_img)
        shutil.copy(label_path, dst_lbl)


def write_data_yaml():
    """
    เขียนไฟล์ data.yaml
    YOLO ชอบ path แบบ forward slash / มากกว่า \
    """
    train_dir = os.path.join(OUTPUT_ROOT, "train", "images").replace("\\", "/")
    val_dir = os.path.join(OUTPUT_ROOT, "val", "images").replace("\\", "/")

    yaml_lines = [
        f"train: {train_dir}",
        f"val: {val_dir}",
        "",
        f"nc: {NUM_CLASSES}",
        "names: [" + ", ".join(f"'{n}'" for n in CLASS_NAMES) + "]",
        "",
    ]
    with open(DATA_YAML_PATH, "w", encoding="utf-8") as f:
        f.write("\n".join(yaml_lines))


def train_yolo():
    """เทรน YOLOv8 ด้วย ultralytics"""
    model = YOLO(BASE_MODEL)
    results = model.train(
        data=DATA_YAML_PATH,
        epochs=50,        # ปรับตามเครื่อง
        imgsz=640,
        batch=16,         # ถ้าแรม/VRAM ไม่พอ ลดเหลือ 8 หรือ 4
        workers=0,        # สำคัญบน Windows/Jupyter
        name="cjmall_yolov8n",
    )
    print("🎉 เทรนเสร็จแล้ว ดูผลที่ runs/detect/cjmall_yolov8n")


def main():
    print("📁 เตรียมโฟลเดอร์ ...")
    ensure_dirs()

    print("🔍 กำลังรวบรวมรูป+label ...")
    pairs = collect_pairs()
    print(f"พบรูปที่มี label คู่กันทั้งหมด: {len(pairs)} รูป")

    if not pairs:
        raise RuntimeError("ไม่พบคู่รูป+txt เลยในโฟลเดอร์ต้นทาง ตรวจสอบ DATASET_ROOT อีกครั้ง")

    train_pairs, val_pairs = split_dataset(pairs, TRAIN_RATIO, RANDOM_SEED)
    print(f"✅ แบ่งข้อมูลแล้ว: train={len(train_pairs)}, val={len(val_pairs)}")

    print("📦 กำลังก็อปปี้ไฟล์ไปยังโครงสร้าง YOLO ...")
    copy_split(train_pairs, "train")
    copy_split(val_pairs, "val")

    print("📝 สร้างไฟล์ data.yaml ...")
    write_data_yaml()
    print(f"✅ สร้างแล้วที่: {DATA_YAML_PATH}")

    print("🚀 เริ่มเทรน YOLO ...")
    train_yolo()


if __name__ == "__main__":
    main()


📁 เตรียมโฟลเดอร์ ...
🔍 กำลังรวบรวมรูป+label ...
พบรูปที่มี label คู่กันทั้งหมด: 52 รูป
✅ แบ่งข้อมูลแล้ว: train=41, val=11
📦 กำลังก็อปปี้ไฟล์ไปยังโครงสร้าง YOLO ...
📝 สร้างไฟล์ data.yaml ...
✅ สร้างแล้วที่: C:\projecteve\OCR\dataset\CJmall_catalog\yolo\data.yaml
🚀 เริ่มเทรน YOLO ...
New https://pypi.org/project/ultralytics/8.3.223 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.161  Python-3.10.18 torch-2.7.1+cpu CPU (12th Gen Intel Core(TM) i7-12650H)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\projecteve\OCR\dataset\CJmall_catalog\yolo\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze

train: Scanning C:\projecteve\OCR\dataset\CJmall_catalog\yolo\train\labels.cache... 51 images, 8 backgrounds, 0 corrupt: 100%|██████████| 51/51 [00:00<?, ?it/s]

val: Fast image access  (ping: 0.00.0 ms, read: 969.01334.3 MB/s, size: 966.8 KB)



c:\Miniconda3\envs\yolov8-env\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\projecteve\OCR\dataset\CJmall_catalog\yolo\val\labels.cache... 23 images, 3 backgrounds, 0 corrupt: 100%|██████████| 23/23 [00:00<?, ?it/s]

Plotting labels to runs\detect\cjmall_yolov8n2\labels.jpg... 


...

      50/50         0G     0.6775     0.3713     0.9516         37        640: 100%|██████████| 4/4 [00:50<00:00, 12.74s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:05<00:00,  5.89s/it]

                   all         23        284      0.999      0.996      0.995      0.838



50 epochs completed in 0.901 hours.
Optimizer stripped from runs\detect\cjmall_yolov8n2\weights\last.pt, 52.0MB
Optimizer stripped from runs\detect\cjmall_yolov8n2\weights\best.pt, 52.0MB

Validating runs\detect\cjmall_yolov8n2\weights\best.pt...
Ultralytics 8.3.161  Python-3.10.18 torch-2.7.1+cpu CPU (12th Gen Intel Core(TM) i7-12650H)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:05<00:00,  5.85s/it]


                   all         23        284      0.998      0.996      0.995      0.839
Speed: 0.8ms preprocess, 235.5ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs\detect\cjmall_yolov8n2
🎉 เทรนเสร็จแล้ว ดูผลที่ runs/detect/cjmall_yolov8n


## 3.crop product

In [1]:
import os
import cv2
from ultralytics import YOLO
from datetime import datetime

# === 1) โหลดโมเดล ===
model = YOLO(r"C:\projecteve\OCR\script\runs\detect\train4\weights\best.pt")

# ดูชื่อคลาสในโมเดลก่อน (เอาไว้ debug)
print("📦 Model classes:", model.names)

# สมมติคลาสที่เทรนคือ 'product'
TARGET_CLASS_NAME = "product"   # 👉 ถ้าใน data.yaml ตั้งชื่ออื่นให้เปลี่ยนตรงนี้

# === 2) พาธรูปต้นฉบับ ===
test_folder = r"C:\projecteve\OCR\data\Go_Wholesale\2025-11-03-18-23\downloaded_pages"

# === 3) โฟลเดอร์เก็บครอป ===
today_str = datetime.today().strftime("%Y-%m-%d")
crop_output_folder = rf"C:\projecteve\OCR\data\yolo_crop\output_detail_only{today_str}"
os.makedirs(crop_output_folder, exist_ok=True)

# === 4) รัน detect ทีเดียวทั้งโฟลเดอร์ ===
# ใส่ save=False เพื่อไม่ให้ไปเขียนทับที่ runs/
results = model.predict(
    source=test_folder,
    imgsz=640,
    conf=0.25,
    save=False,
    verbose=False
)

global_detail_count = 0

for res in results:
    # res.path = พาธรูปต้นฉบับ
    orig_img = cv2.imread(str(res.path))
    if orig_img is None:
        print(f"⚠️ โหลดภาพไม่ได้: {res.path}")
        continue

    # วนทุกกล่องที่ detect ได้
    boxes = res.boxes
    if boxes is None or len(boxes) == 0:
        continue

    for box in boxes:
        # box.xyxy, box.cls
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
        class_id = int(box.cls[0].cpu().numpy())

        # ชื่อคลาสจากโมเดล
        label = model.names[class_id]

        # เช็กว่าใช่คลาสที่เราต้องการไหม
        if label == TARGET_CLASS_NAME:
            global_detail_count += 1
            crop_img = orig_img[y1:y2, x1:x2]

            crop_filename = f"{today_str}_detail_{global_detail_count:04d}.jpg"
            crop_path = os.path.join(crop_output_folder, crop_filename)

            cv2.imwrite(crop_path, crop_img)
            print(f"✅ ครอปและบันทึก: {crop_path}")

print(f"🎉 เสร็จแล้ว ครอปได้ {global_detail_count} รูป ใส่ไว้ที่: {crop_output_folder}")


📦 Model classes: {0: 'product'}
✅ ครอปและบันทึก: C:\projecteve\OCR\data\yolo_crop\output_detail_only2025-11-04\2025-11-04_detail_0001.jpg
✅ ครอปและบันทึก: C:\projecteve\OCR\data\yolo_crop\output_detail_only2025-11-04\2025-11-04_detail_0002.jpg
✅ ครอปและบันทึก: C:\projecteve\OCR\data\yolo_crop\output_detail_only2025-11-04\2025-11-04_detail_0003.jpg
✅ ครอปและบันทึก: C:\projecteve\OCR\data\yolo_crop\output_detail_only2025-11-04\2025-11-04_detail_0004.jpg
✅ ครอปและบันทึก: C:\projecteve\OCR\data\yolo_crop\output_detail_only2025-11-04\2025-11-04_detail_0005.jpg
✅ ครอปและบันทึก: C:\projecteve\OCR\data\yolo_crop\output_detail_only2025-11-04\2025-11-04_detail_0006.jpg
✅ ครอปและบันทึก: C:\projecteve\OCR\data\yolo_crop\output_detail_only2025-11-04\2025-11-04_detail_0007.jpg
✅ ครอปและบันทึก: C:\projecteve\OCR\data\yolo_crop\output_detail_only2025-11-04\2025-11-04_detail_0008.jpg
✅ ครอปและบันทึก: C:\projecteve\OCR\data\yolo_crop\output_detail_only2025-11-04\2025-11-04_detail_0009.jpg
✅ ครอปและบันทึ